In [ ]:
# ==============================================================
# 0. LIBRERÍAS Y CONFIGURACIÓN INICIAL
# ==============================================================

import os
import random
import math

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# --------------------------------------------------------------
# Intentamos importar MLflow (si no está instalado, el código
# sigue funcionando pero sin registrar experimentos)
# --------------------------------------------------------------
try:
    import mlflow
    import mlflow.keras as mlflow_keras
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Aviso: MLflow no está instalado. "
          "El código correrá igual, pero sin registrar experimentos.")

# --------------------------------------------------------------
# Fijamos semillas para reproducibilidad básica
# --------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# ==============================================================
# 1. CARGA DEL CONJUNTO DE DATOS LIMPIO
# ==============================================================

# Ruta al archivo limpio (ajusta el nombre si es necesario)
DATA_PATH = "listings_model_no_outliers.csv"

# Leemos el CSV en un DataFrame
df = pd.read_csv(DATA_PATH)

# Mostramos información básica para verificar que cargó bien
print("Shape del DataFrame:", df.shape)
print("Primeras columnas:", df.columns.tolist()[:20])

# ==============================================================
# 2. DEFINICIÓN DE LA VARIABLE OBJETIVO (REGRESIÓN)
# ==============================================================

# En el proyecto queremos predecir el "precio por noche".
# Buscamos el nombre de la columna de target.
posibles_targets = ["price", "price_night", "price_per_night"]
target_col = None

for col in posibles_targets:
    if col in df.columns:
        target_col = col
        break

if target_col is None:
    raise ValueError(
        "No se encontró ninguna columna de precio en el DataFrame. "
        "Revisa el nombre de la columna de precio y actualiza 'posibles_targets'."
    )

print(f"Usaremos la columna '{target_col}' como variable objetivo (y).")

# Si la columna de precio está como texto con símbolos ($, comas), la convertimos a numérico
if df[target_col].dtype == "O":
    df[target_col] = (
        df[target_col]
        .astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .astype(float)
    )

# Nos aseguramos de que no haya valores nulos en la variable objetivo
df = df[df[target_col].notna()].copy()

# ==============================================================
# 2b. DEFINICIÓN DE ETIQUETA "RECOMENDADO" (PREGUNTA 3)
# ==============================================================

# Nos aseguramos de que existan las columnas necesarias
cols_necesarias = [
    "review_scores_rating",
    "review_scores_cleanliness",
    "number_of_reviews"
]
for c in cols_necesarias:
    if c not in df.columns:
        raise ValueError(
            f"Falta la columna requerida para la regla de recomendación: {c}"
        )

# Creamos la columna 'recomendado' basada en la regla:
# recomendado = 1 si:
#   review_scores_rating       >= 4.0
#   y review_scores_cleanliness >= 4.0
#   y number_of_reviews        >= 5
# recomendado = 0 en caso contrario

mask_valid = (
    df["review_scores_rating"].notna()
    & df["review_scores_cleanliness"].notna()
    & df["number_of_reviews"].notna()
)

df["recomendado"] = 0  # por defecto no recomendado

df.loc[
    mask_valid
    & (df["review_scores_rating"] >= 4.0)
    & (df["review_scores_cleanliness"] >= 4.0)
    & (df["number_of_reviews"] >= 5),
    "recomendado"
] = 1

# Resumen general (para usar en el informe)
total_con_info = mask_valid.sum()
total_recomendados = df.loc[mask_valid, "recomendado"].sum()
porc_recomendados = total_recomendados / total_con_info * 100

print("\n===================================================")
print("CLASIFICACIÓN DE LISTINGS: RECOMENDADOS VS NO RECOMENDADOS")
print("===================================================")
print(f"Listings con info suficiente para aplicar la regla: {total_con_info}")
print(f"Listings recomendados: {total_recomendados} "
      f"({porc_recomendados:.1f}% de los que tienen info suficiente)")
print("La etiqueta 'recomendado' se define por rating >= 4.0, "
      "limpieza >= 4.0 y al menos 5 reseñas.\n")


# ==============================================================
# 3. SELECCIÓN DE VARIABLES PREDICTORAS (X)
# ==============================================================

# a) Eliminamos columnas identificadoras o claramente no útiles para la red
cols_descartar = [
    target_col,
    "id",
    "listing_id",
    "name",
    "description",
    "host_name",
    "host_id",
    "last_review",
    "recomendado",   
]


cols_descartar = [c for c in cols_descartar if c in df.columns]

# Tomamos todas las demás columnas como candidatos
X_raw = df.drop(columns=cols_descartar)

# b) Convertimos columnas booleanas a 0/1 y luego nos quedamos
#    con todas las columnas numéricas (incluye dummies one-hot)
bool_cols = X_raw.select_dtypes(include=["bool"]).columns
print("Columnas booleanas encontradas (se convierten a 0/1):", len(bool_cols))

if len(bool_cols) > 0:
    X_raw[bool_cols] = X_raw[bool_cols].astype(int)

X_num = X_raw.select_dtypes(include=[np.number]).copy()

print("Variables numéricas iniciales:", X_num.shape[1])


# --- c) Detectar columnas sin variación (todas las filas tienen el mismo valor) ---
cols_sin_variacion = X_num.columns[X_num.nunique() <= 1].tolist()
print("Columnas sin variación (se eliminan):", len(cols_sin_variacion))

# --- d) Detectar dummies casi siempre 0 o casi siempre 1 ---
# Identificamos columnas binarias (solo 0 y 1, ignorando NaN)
binary_cols = []
for col in X_num.columns:
    valores = set(X_num[col].dropna().unique())
    if valores.issubset({0, 1}):
        binary_cols.append(col)

# Calculamos proporción de 1s en cada dummy
proporcion_positivos = X_num[binary_cols].mean()

# Umbral: menos de 1% de 1s (o más de 99% de 1s)
umbral = 0.01
cols_casi_todo_cero = proporcion_positivos[proporcion_positivos < umbral].index.tolist()
cols_casi_todo_uno = proporcion_positivos[proporcion_positivos > (1 - umbral)].index.tolist()

print("Dummies casi siempre 0 (se eliminan):", len(cols_casi_todo_cero))
print("Dummies casi siempre 1 (se eliminan):", len(cols_casi_todo_uno))

# Lista final de columnas a eliminar por poca información
cols_eliminar_por_poca_info = list(
    set(cols_sin_variacion + cols_casi_todo_cero + cols_casi_todo_uno)
)

# Creamos la matriz final X eliminando esas columnas
X = X_num.drop(columns=cols_eliminar_por_poca_info).copy()

feature_names = X.columns.tolist()

print("Variables finales usadas como features:", len(feature_names))

# Variable objetivo (serie o vector)
y = df[target_col].values

# ==============================================================
# 4. PARTICIÓN TRAIN / VALIDACIÓN / TEST
# ==============================================================

# Primero hacemos un split train_val vs test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED
)

# Luego, dentro de train_val, separamos train y validación
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.20, random_state=SEED
)

print("Tamaño X_train:", X_train.shape)
print("Tamaño X_val:", X_val.shape)
print("Tamaño X_test:", X_test.shape)

# Convertimos a numpy float32 para evitar problemas con Keras
X_train_np = X_train.to_numpy().astype("float32")
X_val_np   = X_val.to_numpy().astype("float32")
X_test_np  = X_test.to_numpy().astype("float32")

# ==============================================================
# 5. CAPA DE NORMALIZACIÓN PARA LA RED NEURONAL
# ==============================================================

# Usamos la capa Normalization de Keras
normalizer = keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train_np)   # ahora sí, con numpy

input_dim = X_train_np.shape[1]

# ==============================================================
# 6. FUNCIÓN PARA CONSTRUIR EL MODELO DE REGRESIÓN
# ==============================================================

def build_regression_model(
    input_dim,
    n_layers=2,
    n_units=64,
    learning_rate=1e-3,
):
    """
    Construye un modelo de red neuronal para regresión:
    - Capa de entrada + capa de normalización
    - n_layers capas densas ocultas con activación ReLU
    - 1 capa de salida con activación lineal (predicción de precio)
    """
    model = keras.Sequential(name="nn_regression_price")

    # Capa de entrada + normalización
    model.add(keras.layers.Input(shape=(input_dim,), name="input"))
    model.add(normalizer)

    # Capas ocultas (no lineales, ReLU)
    for i in range(n_layers):
        model.add(
            keras.layers.Dense(
                n_units,
                activation="relu",
                name=f"dense_hidden_{i+1}"
            )
        )

    # Capa de salida (lineal) para regresión
    model.add(
        keras.layers.Dense(
            1,
            activation="linear",
            name="output_price"
        )
    )

    # Definimos el optimizador Adam con el learning rate dado
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    # Compilamos el modelo con loss de MSE y métricas MAE y MSE
    model.compile(
        optimizer=optimizer,
        loss="mse",
        metrics=["mae", "mse"]
    )

    return model

# ==============================================================
# 7. BÚSQUEDA DE HIPERPARÁMETROS (GRID PEQUEÑO)
# ==============================================================

# Listas para la búsqueda (puedes ajustarlas si se demora mucho)
n_layers_list = [1, 2, 3]         # número de capas ocultas
n_units_list = [32, 64, 128]      # neuronas por capa
learning_rates = [1e-3, 5e-4]     # tasas de aprendizaje
batch_sizes = [64, 128]           # tamaño de batch
max_epochs = 80                   # máximo de épocas

# EarlyStopping para evitar sobreentrenamiento
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

# Variables para guardar el mejor modelo y sus resultados
best_model = None
best_history = None
best_val_mae = np.inf
best_params = {}

# Si MLflow está disponible, definimos el experimento
if MLFLOW_AVAILABLE:
    mlflow.set_experiment("airbnb_regresion_redes_neuronales")
# --------------------------------------------------------------
# Recorremos todas las combinaciones de hiperparámetros
# --------------------------------------------------------------
from itertools import product

run_id = 0

# Lista donde vamos a ir guardando resultados de cada combinación
resultados_hp = []   # cada elemento será un diccionario

for n_layers, n_units, lr, batch in product(
    n_layers_list, n_units_list, learning_rates, batch_sizes
):
    run_id += 1
    print("\n===================================================")
    print(f"Ejecutando combinación #{run_id}:")
    print(f"  n_layers = {n_layers}, n_units = {n_units}, "
          f"lr = {lr}, batch_size = {batch}")
    print("===================================================")

    # Creamos el modelo
    model = build_regression_model(
        input_dim=input_dim,
        n_layers=n_layers,
        n_units=n_units,
        learning_rate=lr
    )

    # Iniciamos run de MLflow si está disponible
    if MLFLOW_AVAILABLE:
        # 👇 NUEVO: cerrar cualquier run activo antes de abrir uno nuevo
        active_run = mlflow.active_run()
        if active_run is not None:
            mlflow.end_run()

        mlflow.start_run(run_name=f"reg_nn_run_{run_id}")
        mlflow.log_param("model_type", "regression_nn")
        mlflow.log_param("n_layers", n_layers)
        mlflow.log_param("n_units", n_units)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("batch_size", batch)

    # Entrenamos el modelo
    history = model.fit(
        X_train_np, y_train,
        validation_data=(X_val_np, y_val),
        epochs=max_epochs,
        batch_size=batch,
        verbose=0,
        callbacks=[early_stop]
    )

    # Obtenemos el mejor MAE de validación
    val_mae_history = history.history["val_mae"]
    min_val_mae = float(np.min(val_mae_history))

    # También podemos mirar el mejor MSE de validación
    val_mse_history = history.history["val_mse"]
    min_val_mse = float(np.min(val_mse_history))

    print(f"Mejor val_MAE en esta corrida: {min_val_mae:.4f}")
    print(f"Mejor val_MSE en esta corrida: {min_val_mse:.4f}")

    # ----------------------------------------------------------
    # Guardamos esta combinación y su desempeño en la lista
    # ----------------------------------------------------------
    resultados_hp.append({
        "run_id": run_id,
        "n_layers": n_layers,
        "n_units": n_units,
        "learning_rate": lr,
        "batch_size": batch,
        "best_val_mae": min_val_mae,
        "best_val_mse": min_val_mse
    })

    # Loggeamos métricas y modelo en MLflow (versión simple, robusta)
    if MLFLOW_AVAILABLE:
        mlflow.log_metric("best_val_mae", min_val_mae)
        mlflow.log_metric("best_val_mse", min_val_mse)
        mlflow_keras.log_model(model, artifact_path="model")
        mlflow.end_run()

    # Actualizamos el mejor modelo si esta corrida es mejor
    if min_val_mae < best_val_mae:
        best_val_mae = min_val_mae
        best_model = model
        best_history = history
        best_params = {
            "n_layers": n_layers,
            "n_units": n_units,
            "learning_rate": lr,
            "batch_size": batch
        }
        print(">>> ACTUALIZANDO mejor modelo con esta combinación.")

# ==============================================================
# 7b. TABLA RESUMEN DE LA BÚSQUEDA DE HIPERPARÁMETROS
# ==============================================================

df_resultados_hp = pd.DataFrame(resultados_hp)

# Ordenamos por mejor val_MAE (menor es mejor)
df_resultados_hp = df_resultados_hp.sort_values("best_val_mae", ascending=True)

print("\n===================================================")
print("RESUMEN DE TODAS LAS COMBINACIONES PROBADAS (TOP 10)")
print("Ordenado por mejor val_MAE (menor es mejor)")
print("===================================================\n")

print(df_resultados_hp.head(10))

# Si quieres, también puedes guardar la tabla completa a CSV
df_resultados_hp.to_csv("resultados_hiperparametros_regresion_nn.csv", index=False)
print("\nTabla completa guardada en 'resultados_hiperparametros_regresion_nn.csv'")

# ==============================================================
# 8. EVALUACIÓN DEL MEJOR MODELO EN CONJUNTO DE TEST
# ==============================================================

if best_model is None:
    raise RuntimeError(
        "No se entrenó ningún modelo. Revisa la definición de la búsqueda."
    )

# Predicciones en el conjunto de prueba
y_pred_test = best_model.predict(X_test_np).flatten()

# Cálculo de métricas de desempeño
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)
test_rmse = math.sqrt(test_mse)
test_r2 = r2_score(y_test, y_pred_test)

# ==============================================================
# 8b. GRÁFICO: PRECIO REAL vs PRECIO PREDICHO (SERIE)
# ==============================================================

df_pred_vs_real = pd.DataFrame({
    "precio_real": y_test,
    "precio_predicho": y_pred_test
}).reset_index(drop=True)

plt.figure(figsize=(10, 5))
plt.plot(df_pred_vs_real["precio_real"].values, label="Precio real")
plt.plot(df_pred_vs_real["precio_predicho"].values, label="Precio predicho", alpha=0.8)
plt.xlabel("Observación en test")
plt.ylabel("Precio")
plt.title("Precio real vs precio predicho - Conjunto de test (serie)")
plt.legend()
plt.tight_layout()
plt.show()
# plt.savefig("precio_real_vs_predicho_serie_test.png", dpi=300)

# ==============================================================
# 8c. GRÁFICO: PRECIO REAL vs PRECIO PREDICHO (SCATTER 45°)
# ==============================================================

plt.figure(figsize=(6, 6))
plt.axes(aspect='equal')
plt.scatter(y_test, y_pred_test, alpha=0.6)
plt.xlabel("Precio real")
plt.ylabel("Precio predicho")
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
lims = [min_val, max_val]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, linestyle="--")
plt.title("Dispersión precio real vs predicho - Conjunto de test")
plt.tight_layout()
plt.show()
# plt.savefig("precio_real_vs_predicho_scatter_test.png", dpi=300)

# ==============================================================
# 8d. HISTORIAL DE PÉRDIDA Y MAE DEL MEJOR MODELO
# ==============================================================

if best_history is not None:
    history_dict = best_history.history

    # Historial de pérdida (loss)
    plt.figure(figsize=(8, 5))
    plt.plot(history_dict["loss"], label="Pérdida entrenamiento")
    plt.plot(history_dict["val_loss"], label="Pérdida validación")
    plt.xlabel("Época")
    plt.ylabel("Loss (MSE)")
    plt.title("Historial de pérdida - Mejor modelo")
    plt.legend()
    plt.tight_layout()
    plt.show()
    # plt.savefig("historial_perdida_mejor_modelo.png", dpi=300)

    # Historial de MAE
    plt.figure(figsize=(8, 5))
    plt.plot(history_dict["mae"], label="MAE entrenamiento")
    plt.plot(history_dict["val_mae"], label="MAE validación")
    plt.xlabel("Época")
    plt.ylabel("MAE")
    plt.title("Historial de MAE - Mejor modelo")
    plt.legend()
    plt.tight_layout()
    plt.show()
    # plt.savefig("historial_mae_mejor_modelo.png", dpi=300)

else:
    print("No se guardó el history del mejor modelo, "
          "no se puede graficar el historial de pérdida/MAE.")

# ==============================================================
# 8e. ANÁLISIS DE CORRELACIONES PARA GENERAR RECOMENDACIONES
# ==============================================================

# Unimos X y y para calcular correlaciones simples con el precio
df_corr = pd.concat(
    [X.reset_index(drop=True), pd.Series(y, name=target_col).reset_index(drop=True)],
    axis=1
)

corr_with_price = df_corr.corr()[target_col].drop(target_col).sort_values(ascending=False)

print("\n===================================================")
print("CORRELACIONES DE FEATURES CON EL PRECIO")
print("===================================================")

print("\nTop 10 variables con correlación POSITIVA con el precio:")
for feat, val in corr_with_price.head(10).items():
    print(f"  {feat:30s} -> corr = {val:.3f}")

print("\nTop 10 variables con correlación NEGATIVA con el precio:")
for feat, val in corr_with_price.tail(10).items():
    print(f"  {feat:30s} -> corr = {val:.3f}")

print("\nUsa estas correlaciones para derivar recomendaciones a anfitriones en el reporte.")
print("Por ejemplo: enfocarse en las variables fuertemente positivas para justificar precios más altos.")

# ==============================================================
# 9. GRÁFICO DE LA ARQUITECTURA DEL MEJOR MODELO
# ==============================================================

try:
    keras.utils.plot_model(
        best_model,
        to_file="arquitectura_mejor_modelo.png",
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True
    )
    print("\nDiagrama de arquitectura guardado en 'arquitectura_mejor_modelo.png'.")
    print("Revisa este archivo PNG para incluirlo en tu reporte.")
except Exception as e:
    print("\nNo se pudo graficar la arquitectura del modelo.")
    print("Posibles causas: falta de graphviz/pydot. Error original:")
    print(e)

# ==============================================================
# 10. RESUMEN FINAL DEL MEJOR MODELO
# ==============================================================

print("\n\n===================================================")
print("RESUMEN DEL MEJOR MODELO DE REGRESIÓN (RED NEURONAL)")
print("===================================================\n")

print(">> Hiperparámetros seleccionados:")
for k, v in best_params.items():
    print(f"   {k}: {v}")

print("\n>> Métricas en VALIDACIÓN (mejor valor durante el entrenamiento):")
print(f"   Mejor val_MAE: {best_val_mae:.4f}")

print("\n>> Métricas en TEST (conjunto hold-out):")
print(f"   MAE  (test): {test_mae:.4f}")
print(f"   MSE  (test): {test_mse:.4f}")
print(f"   RMSE (test): {test_rmse:.4f}")
print(f"   R^2  (test): {test_r2:.4f}")

# --------------------------------------------------------------
# Información sobre arquitectura: capas, activaciones, etc.
# --------------------------------------------------------------

print("\n>> Arquitectura del modelo (best_model.summary()):\n")
best_model.summary()

# Contamos cuántas capas densas lineales y no lineales hay
num_dense_layers = 0
num_linear_layers = 0
num_nonlinear_layers = 0
activations = []

for layer in best_model.layers:
    if isinstance(layer, keras.layers.Dense):
        num_dense_layers += 1
        act = layer.activation.__name__ if hasattr(layer, "activation") else "none"
        activations.append((layer.name, act))
        if act == "linear":
            num_linear_layers += 1
        else:
            num_nonlinear_layers += 1

print("\n>> Detalle de capas densas y activaciones:")
for name, act in activations:
    print(f"   Capa: {name:20s} | Activación: {act}")

print("\n>> Resumen de capas densas:")
print(f"   Total capas densas: {num_dense_layers}")
print(f"   Capas con activación NO lineal (ej. ReLU): {num_nonlinear_layers}")
print(f"   Capas con activación lineal (incluye la de salida): {num_linear_layers}")

# --------------------------------------------------------------
# Información sobre variables usadas
# --------------------------------------------------------------

print("\n>> Variables de entrada utilizadas (features):")
print(f"   Número total de variables: {len(feature_names)}")
print("   Algunas de ellas (primeras 30):")
for f in feature_names[:30]:
    print("    -", f)

print("\nFin del script de regresión con redes neuronales para Airbnb.")
print("Este modelo está pensado para el cliente 'viajeros - clientes frecuentes',")
print("ya que permite estimar el precio por noche a partir de múltiples características ")
print("de la propiedad y del host.")
